In [2]:
import nltk
import h5py
import pickle
from nltk.corpus import stopwords
from scipy.sparse import *
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import numpy as np
from numpy.random import RandomState
from sklearn.feature_extraction.text import CountVectorizer
import keras.preprocessing.text as kpt
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Activation, Dense, Input, Flatten, Dropout, Conv1D, GlobalMaxPooling1D, Embedding, AveragePooling1D
from keras.models import Sequential
from keras.utils.np_utils import to_categorical
from keras.callbacks import ModelCheckpoint
import pandas as pd
from sklearn.utils import shuffle
import re
from collections import Counter
from gensim.models.word2vec import Word2Vec
import itertools
import csv
import keras
from nltk.stem import PorterStemmer

ps = PorterStemmer()

lemmatizer = WordNetLemmatizer()
vectorizer = CountVectorizer()
stopWords = set(stopwords.words('english'))

Using TensorFlow backend.


In [124]:
trainData = pd.read_csv("sentiment140train.csv", encoding = 'latin1')
testData = pd.read_csv("sentiment140test.csv", encoding = 'latin1')

In [4]:
trainData.columns = ['sentiment', 'id', 'date', 'q', 'user', 'tweet']
trainData = trainData.drop(['id', 'date', 'q', 'user'], axis=1)
trainData = shuffle(trainData)
trainY = trainData.sentiment
trainX = trainData.tweet

In [5]:
testData.columns = ['sentiment', 'id', 'date', 'subject', 'user', 'tweet']
testData = testData.drop(['id', 'date', 'subject', 'user'], axis=1)
testY = testData.sentiment
testX = testData.tweet

In [6]:
###
cleanTestX = list()
for x in testX : 
    cleanTestX.append((''.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)","",x))))

In [7]:
cleanTrainX = list()
for x in trainX :
    cleanTrainX.append((''.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",x))))

In [8]:
testY = pd.Series.tolist(testY)
trainY = pd.Series.tolist(trainY)

In [119]:
testData.sentiment = pd.Series.tolist(testData.sentiment)

In [9]:
for i in range(len(testY)) :
    j = testY[i]
    if j == 4 :
        testY[i] = 1
    elif j== 2 :
        testY[i] = 1

In [10]:
set(testY)

{0, 1}

In [11]:
for i in range(len(trainY)) :
    j = trainY[i]
    if j == 4 :
        trainY[i] = 1
    elif j== 2 :
        trainY[i] = 0.5

In [12]:
set(trainY)

{0, 1}

In [13]:
cleanTestX

['Reading my kindle2  Love it Lee childs is good read',
 'Ok first assesment of the kindle2 it fucking rocks',
 ' Youll love your Kindle2 Ive had mine for a few months and never looked back The new big one is huge No need for remorse ',
 '  Fair enough But i have the Kindle2 and I think its perfect  ',
 ' no it is too big Im quite happy with the Kindle2',
 'Fuck this economy I hate aig and their non loan given asses',
 'Jquery is my new best friend',
 'Loves twitter',
 'how can you not love Obama he makes jokes about himself',
 'Check this video out  President Obama at the White House Correspondents Dinner ',
 ' I firmly believe that ObamaPelosi have ZERO desire to be civil  Its a charade and a slogan but they want to destroy conservatism',
 'House Correspondents dinner was last night whoopi barbara amp sherri went Obama got a standing ovation',
 'Watchin EspnJus seen this new Nike Commerical with a Puppet Lebronsht was hilariousLMAO',
 'dear nike stop with the flywire that shit is a w

In [14]:
#ATTENTION.. YOU HAVE ALREADY SAVED purestLexicon.pkl file.. load it directly and avoid all the steps below to find interesting words
lexicon = []
for i in cleanTrainX :
    allWords = word_tokenize(i)
    lexicon += allWords
for i in cleanTestX :
    allWords = word_tokenize(i)
    lexicon += allWords

In [15]:
# with open('lexicon.pkl', 'wb') as f :
#     pickle.dump(lexicon, f)

In [16]:
# with open('lexicon.pkl', 'rb') as f:
#     lexicon = pickle.load(f)

In [17]:
len(lexicon)

21139328

In [18]:
## EXPERIMENT PENDING
simplifiedLexicon = [x.lower() for x in lexicon if len(x)>1]
len(simplifiedLexicon)

18950991

In [19]:
simplifiedLexicon

['new',
 'post',
 'read',
 'about',
 'my',
 'adventures',
 'as',
 'slut',
 'wife',
 'already',
 'went',
 'to',
 'the',
 'grocery',
 'store',
 'they',
 're',
 'calling',
 'for',
 'thunderstorms',
 'today',
 'though',
 'is',
 'in',
 'karlsta',
 'in',
 'my',
 'summerhouse',
 'with',
 'my',
 'family',
 'and',
 'it',
 'raining',
 'of',
 'angels',
 'only',
 'once',
 'oh',
 'got',
 'ta',
 'love',
 'miami',
 'amp',
 'vegas',
 'but',
 'don',
 'know',
 'anybody',
 'there',
 'thinks',
 'twitter',
 'is',
 'over',
 'ratedd',
 'but',
 'is',
 'willing',
 'to',
 'give',
 'it',
 'goo',
 'morning',
 'have',
 'great',
 'monday',
 'off',
 'to',
 'work',
 'which',
 'means',
 'you',
 'can',
 'totally',
 'call',
 'me',
 'today',
 'though',
 'please',
 'don',
 'do',
 'it',
 'super',
 'early',
 'in',
 'the',
 'morning',
 'tiiiiiired',
 'thank',
 'you',
 'all',
 'so',
 'much',
 'for',
 'the',
 'birthday',
 'wishes',
 'it',
 'was',
 'really',
 'touching',
 'to',
 'hear',
 'from',
 'you',
 'oh',
 'thats',
 'rubbi

In [20]:
stopWords

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [21]:
pureLexicon = [i for i in simplifiedLexicon if not i in stopWords]
len(pureLexicon)

11264288

In [22]:
pureLexicon

['new',
 'post',
 'read',
 'adventures',
 'slut',
 'wife',
 'already',
 'went',
 'grocery',
 'store',
 'calling',
 'thunderstorms',
 'today',
 'though',
 'karlsta',
 'summerhouse',
 'family',
 'raining',
 'angels',
 'oh',
 'got',
 'ta',
 'love',
 'miami',
 'amp',
 'vegas',
 'know',
 'anybody',
 'thinks',
 'twitter',
 'ratedd',
 'willing',
 'give',
 'goo',
 'morning',
 'great',
 'monday',
 'work',
 'means',
 'totally',
 'call',
 'today',
 'though',
 'please',
 'super',
 'early',
 'morning',
 'tiiiiiired',
 'thank',
 'much',
 'birthday',
 'wishes',
 'really',
 'touching',
 'hear',
 'oh',
 'thats',
 'rubbish',
 'know',
 'means',
 'sounds',
 'bad',
 'without',
 'long',
 'hope',
 'folks',
 'leaving',
 'today',
 'got',
 'toilet',
 'started',
 'walking',
 'almost',
 'broke',
 'neck',
 'foot',
 'fell',
 'asleep',
 'hahaha',
 'thank',
 'gon',
 'na',
 'loooooooooooooong',
 'wait',
 'till',
 'next',
 'thursday',
 'followfriday',
 'thanks',
 'day',
 'making',
 'pimples',
 'go',
 'away',
 'still',


In [123]:
stopWords = stopWords - {'not', 'no', 'yes', 'yo', 'nor'}

In [23]:
stopWords

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [24]:
#remove stop words from lexicon

In [25]:
wCounts = Counter(pureLexicon)
wCounts.most_common()

[('good', 91367),
 ('day', 87114),
 ('get', 82068),
 ('like', 78589),
 ('go', 73950),
 ('quot', 72085),
 ('got', 70619),
 ('today', 68237),
 ('work', 64935),
 ('love', 64869),
 ('going', 64633),
 ('lol', 59282),
 ('time', 58002),
 ('back', 56964),
 ('one', 54090),
 ('know', 52059),
 ('im', 50683),
 ('really', 50044),
 ('amp', 48705),
 ('see', 46391),
 ('night', 43648),
 ('still', 43575),
 ('well', 42796),
 ('new', 42490),
 ('want', 42308),
 ('na', 42093),
 ('think', 41414),
 ('home', 40451),
 ('thanks', 40333),
 ('oh', 39800),
 ('much', 37053),
 ('miss', 36446),
 ('need', 35974),
 ('last', 35743),
 ('morning', 34606),
 ('tomorrow', 34127),
 ('hope', 33903),
 ('great', 33546),
 ('twitter', 32773),
 ('haha', 31514),
 ('feel', 30275),
 ('sad', 29432),
 ('fun', 28700),
 ('wish', 28291),
 ('sleep', 27989),
 ('right', 27891),
 ('bad', 27341),
 ('would', 27336),
 ('happy', 27053),
 ('sorry', 26379),
 ('tonight', 26017),
 ('come', 25125),
 ('make', 24971),
 ('way', 24476),
 ('getting', 24368),

In [26]:
## EXPERIMENT PENDING
## removing most common or not so common words
#since the max count is still too much.. we'll start with selecting words with only count between 1k and 10k.

purestLexicon = []
for w in wCounts :
#     print(wCounts[w])
    if wCounts[w]>1000:
        purestLexicon.append(w)


In [27]:
len(purestLexicon)
#the number was 1497 with stemmed words

1520

In [28]:
with open("purestLexicon2.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(purestLexicon)

In [29]:
#simplifying text
testXFinalized = []
for i in cleanTestX :
    allWords = word_tokenize(i)
#     print(allWords)
    temp = ""
    for x in allWords:
        newWord = x.lower()
        if newWord in purestLexicon :
#             print (newWord)
            temp = temp + " " + newWord
#     print (temp)
#             temp.append((''.join(newWord)))
#             temp += (newWord)
    testXFinalized.append(temp)
    
#--------------------------------------------#

In [30]:
testXFinalized

[' reading love good read',
 ' ok first fucking rocks',
 ' love ive mine months never looked back new big one huge need',
 ' fair enough think perfect',
 ' big im quite happy',
 ' fuck hate non given',
 ' new best friend',
 ' loves twitter',
 ' love makes',
 ' check video white house dinner',
 ' believe want',
 ' house dinner last night amp went got',
 ' watchin seen new',
 ' dear stop shit science ugly love',
 ' best time dont want get 12',
 ' talking guy last night telling die hard fan also told hates james',
 ' love',
 ' im still end',
 ' boss',
 ' lol love lakers lets go lol',
 ' awesome',
 ' nobody comes even close',
 ' iphone much fun app anything',
 ' good news call office saying everything sick',
 ' awesome come back via',
 ' long weekend much needed',
 ' bad ass social way cool',
 '',
 ' use love much video',
 ' need good got pls dm',
 ' checked google business shows second huh good',
 ' google always good place look worked dad',
 ' played google phone screen would break fast 

In [31]:
#simplifying text
trainXFinalized = []
for i in cleanTrainX :
    allWords = word_tokenize(i)
#     print(allWords)
    temp = ""
    for x in allWords:
        newWord = x.lower()
        if newWord in purestLexicon :
#             print (newWord)
            temp = temp + " " + newWord
#     print (temp)
#             temp.append((''.join(newWord)))
#             temp += (newWord)
    trainXFinalized.append(temp)


In [32]:
with open("simplifiedTrainData14000.csv", "w") as g:
    writer = csv.writer(g)
    writer.writerows(trainXFinalized)

In [33]:
#everything working perfect till here.. don't fuck now!
###############################################
###############################################

In [34]:
#vectorizing these two babies
TR = vectorizer.fit_transform(trainXFinalized)
TE = vectorizer.fit_transform(testXFinalized) 

In [56]:
x_train = TR.toarray()
x_test = TE.toarray()
y_train = np.asarray(trainY)
# y_train = trainY.toarray()
# y_test = testY.toarray()
y_test = np.asarray(testY)

In [57]:
set(y_train)

{0, 1}

In [58]:
set(y_test)

{0, 1}

In [59]:
###############################################
#perfect.. the data is prepared now
x_train.shape

(1599999, 1520)

In [60]:
max_features = 1000
batch_size = 32
maxlen = 100
embedding_dims = 150
filters = 250
kernel_size = 3
hidden_dims = 5000
epochs = 2

# y_train_temp_new = keras.utils.to_categorical(y_train_temp, 3)

In [61]:
x_train_temp = x_train[1:5000]
y_train_temp = y_train[1:5000]
print(x_train_temp.shape)
y_train_temp.shape

(4999, 1520)


(4999,)

In [62]:
print(len(x_train_temp), 'train sequences')
print(len(x_test), 'test sequences')
print('Pad sequences (samples x time)')
x_train_temp = sequence.pad_sequences(x_train_temp, maxlen = maxLen)
x_test = sequence.pad_sequences(x_test, maxlen=maxLen)
print('x_train_temp shape:', x_train.shape)
print('x_test shape:', x_test.shape)


4999 train sequences
497 test sequences
Pad sequences (samples x time)
x_train_temp shape: (1599999, 1520)
x_test shape: (497, 100)


In [109]:
model = Sequential()
model.add(Embedding(max_features, embedding_dims, input_length=maxlen))
model.add(Conv1D(filters, kernel_size, padding='same', activation='relu', strides = 1))
model.add(GlobalMaxPooling1D())
model.add(keras.layers.Dropout(0.3))
model.add(Conv1D(filters, kernel_size, padding='same', activation='relu', strides = 1))
model.add(GlobalMaxPooling1D())
model.add(keras.layers.Dropout(0.3))
model.add(Conv1D(filters=32, kernel_size=5, padding='same', activation='relu', strides = 1))
model.add(GlobalMaxPooling1D())
model.add(keras.layers.Dropout(0.3))
model.add(Conv1D(filters=32, kernel_size=7, padding='same', activation='relu', strides = 1))
model.add(GlobalMaxPooling1D())
model.add(keras.layers.Dropout(0.3))
model.add(Conv1D(filters=32, kernel_size=9, padding='same', activation='relu', strides = 1))
model.add(GlobalMaxPooling1D())
model.add(keras.layers.Dropout(0.3))
model.add(Conv1D(filters=32, kernel_size=12, padding='same', activation='relu', strides = 1))
model.add(GlobalMaxPooling1D())
model.add(keras.layers.Dropout(0.3))
model.add(Conv1D(filters=32, kernel_size=15, padding='same', activation='relu', strides = 1))
model.add(GlobalMaxPooling1D())
model.add(keras.layers.Dropout(0.3))
model.add(LSTM(200,dropout=0.3, recurrent_dropout=0.3))
model.add(Dense(labels_count, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())


ValueError: Input 0 is incompatible with layer conv1d_49: expected ndim=3, found ndim=2

In [ ]:
# model = Sequential()
# model.add(Embedding(max_features,
#                  output_dim = embedding_dims,
#                  input_length = maxLen))
# model.add(Dropout(dropout_prob[0]))
# model.add(graph)
# model.add(Dense(128))
# model.add(Dropout(dropout_prob[1]))
# model.add(Activation('relu'))
# model.add(Dense(2))
# model.add(Activation('sigmoid'))

# number_of_epochs = 50

In [116]:
model1 = Sequential()
model1.add(Embedding(max_features, embedding_dims, input_length=maxlen))
model1.add(Conv1D(filters, kernel_size, padding='same', activation='relu', strides = 1))

model1.add(Conv1D(filters, kernel_size, padding='same', activation='relu', strides = 1))

model1.add(Conv1D(filters, kernel_size, padding='same', activation='relu', strides = 1))

model1.add(Conv1D(filters, kernel_size, padding='same', activation='relu', strides = 1))
model1.add(GlobalMaxPooling1D())
# model1.add(Flatten())
model1.add(Dense(250, activation='relu'))
model1.add(Dense(1, activation='sigmoid'))
model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [117]:
print(model1.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_43 (Embedding)     (None, 100, 150)          150000    
_________________________________________________________________
conv1d_56 (Conv1D)           (None, 100, 250)          112750    
_________________________________________________________________
conv1d_57 (Conv1D)           (None, 100, 250)          187750    
_________________________________________________________________
conv1d_58 (Conv1D)           (None, 100, 250)          187750    
_________________________________________________________________
conv1d_59 (Conv1D)           (None, 100, 250)          187750    
_________________________________________________________________
conv1d_60 (Conv1D)           (None, 100, 250)          187750    
_________________________________________________________________
conv1d_61 (Conv1D)           (None, 100, 250)          187750    
__________

In [118]:
model1.fit(x_train_temp, y_train_temp, validation_data=(x_test, y_test), epochs=2, batch_size=128, verbose=2)


Train on 4999 samples, validate on 497 samples
Epoch 1/2
 - 146s - loss: 0.6950 - acc: 0.4991 - val_loss: 0.6899 - val_acc: 0.6439
Epoch 2/2
 - 138s - loss: 0.6932 - acc: 0.5037 - val_loss: 0.6893 - val_acc: 0.6439


In [85]:
print('Build model...')
model = Sequential()

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
model.add(Embedding(max_features,
                    embedding_dims,
                    input_length=maxlen))

model.add(Dropout(0.2))

model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))

model.add(GlobalMaxPooling1D())
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Conv1D(filters, kernel_size, activation='relu', strides = 1))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.add(Dropout(0.3))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Build model...


TypeError: 'int' object is not iterable

In [66]:
model.fit(x_train_temp, y_train_temp,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test))
     

Train on 4999 samples, validate on 497 samples
Epoch 1/2
4999/4999 [==============================] - 30s 6ms/step - loss: 0.6926 - acc: 0.5231 - val_loss: 0.6965 - val_acc: 0.5392
Epoch 2/2
4999/4999 [==============================] - 28s 6ms/step - loss: 0.6922 - acc: 0.5227 - val_loss: 0.6902 - val_acc: 0.5412


In [ ]:
# model.add(Convolution2D(nb_feature_maps, 1, n_gram, wv_sz))
# model.add(MaxPooling2D(poolsize=(nb_tokens - n_gram + 1, 1)))
# model.add(Flatten())
# model.add(WhateverLayer(nb_feature_maps, nb_outputs))
# model.add(...)
 
# ngram_filters = [3, 4, 5, 6, 7, 8]
# conv_filters = []

# for n_gram in ngram_filters:
#     conv_filters.append(Sequential())
#     conv_filters[-1].add(Convolution2D(nb_feature_maps, 1, n_gram, wv_sz))
#     conv_filters[-1].add(MaxPooling2D(poolsize=(nb_tokens - n_gram + 1, 1)))
#     conv_filters[-1].add(Flatten())

# model = Sequential()
# model.add(Merge(conv_filters, mode='concat'))
# model.add(Dropout(0.5))
# model.add(Dense(nb_feature_maps * len(ngram_filters), 1))
# model.add(Activation('sigmoid'))

In [ ]:
predictions = model3.predict(x_test)

In [90]:
errors = predictions.round().flatten()

In [96]:
len(errors[errors==0])

241

In [97]:
len(errors)

497

In [98]:
errors = y_test-predictions.round().flatten()

In [102]:
len(errors[errors==0])

196

In [100]:
len(errors)

497

In [ ]:
len

In [84]:
words = word_tokenize(new_text)

for w in words:
    print(ps.stem(w))

It
is
import
to
by
veri
pythonli
while
you
are
python
with
python
.
all
python
have
python
poorli
at
least
onc
.


In [11]:
ab = " "
listnew += [ps.stem(w) for w in words]

In [12]:
listnew

['It',
 'is',
 'import',
 'to',
 'by',
 'veri',
 'pythonli',
 'while',
 'you',
 'are',
 'python',
 'with',
 'python',
 '.',
 'all',
 'python',
 'have',
 'python',
 'poorli',
 'at',
 'least',
 'onc',
 '.']